# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

import pandas_datareader.data as pdr
!pip install yfinance
import yfinance as yf

import matplotlib.pyplot as plt
plt.style.use('seaborn')

#Import Data

In [ ]:
start = datetime(2021, 4, 1)
end = datetime(2021, 8, 31)

stock = yf.Ticker('AAPL').history(start=start, end=end) #Apple Inc.
market = yf.Ticker('SPY').history(start=start, end=end) #S&P 500 index. Broad market value weighted index that affects all stocks
vix = yf.Ticker('VXX').history(start=start, end=end)   #Volatility index. The 30 day expected volatility of the market implied by option prices on the S&P 500
dxy = yf.Ticker('UUP').history(start=start, end=end) #Dollar index. A weighted index of the value of the US dollar relative to a basket of major currencies
junk = yf.Ticker('JNK').history(start=start, end=end)   #Junk bond index. An index of high yielding/risky corporate bonds

#Train Model

In [ ]:
#Creat target dataframe
target = pd.DataFrame()
target['return'] = stock['Close'].pct_change(1)*100 #data lagged by a day to avoid look-ahead bias
target = target.dropna() #Drop NA in the first row
target.head()

,return
Date,
2021-04-05,2.357733
2021-04-06,0.246223
2021-04-07,1.339037
2021-04-08,1.923379
2021-04-09,2.025158


In [ ]:
#Create features dataframe
features = pd.DataFrame()
features['market'] = market['Close'].pct_change(1)*100
features['vix'] = vix['Close'].diff() #VIX is volatility index and is measured in percentage terms so you only need to take the difference
features['dxy'] = dxy['Close'].pct_change(1)*100
features['junk'] = junk['Close'].pct_change(1)*100
features = features.dropna() #Drop NA in the first row
features.head()

,market,vix,dxy,junk
Date,,,,
2021-04-05,1.435314,-1.119999,-0.319872,0.092059
2021-04-06,-0.059056,0.000000,-0.240672,0.174728
2021-04-07,0.115724,-1.120003,0.120622,0.009176
2021-04-08,0.474680,-0.639999,-0.441762,0.036724
2021-04-09,0.727018,-0.079998,0.080672,-0.082586


In [ ]:
#Run multiple linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

regression = LinearRegression()
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state=0)

#Model is trained on 75% of the data
model = regression.fit(features_train, target_train)

print("Model Intercept:", model.intercept_)
print("Model Coefficients:", model.coef_)

Model Intercept: [-0.01184251]
Model Coefficients: [[ 1.9745785   0.23674555 -0.0595629  -0.21642663]]


#Evaluate Model

In [ ]:
#Use default R2 performance metric
print("Training score: ", model.score(features_train, target_train))
print("Test score: ", model.score(features_test, target_test))

Training score:  0.5442502340459178
Test score:  0.47473771464878267


#Lasso Regression/L1 Regularization

In [ ]:
#Remove less informative features with Lasso Regression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() #Scales input data so that it is has a mean of zero and variance of one
features_standardized = scaler.fit_transform(features) #Every feature now has zero mean and unit variance
lasso_regression = Lasso (alpha = 0.2) #alpha is a hyperparameter (default = 1). Increasing alpha increases regularization

features_standardized_train, features_standardized_test, target_train, target_test = train_test_split(features_standardized, target, test_size=0.25, random_state=0)
lasso_model = lasso_regression.fit(features_standardized_train, target_train)

print("Lasso model intercept:", lasso_model.intercept_)
print("Lasso model coefficients:", lasso_model.coef_)

print("Lasso training score: ", lasso_model.score(features_standardized_train, target_train))
print("Lasso test score: ", lasso_model.score(features_standardized_test, target_test))

Lasso model intercept: [0.19120591]
Lasso model coefficients: [ 0.76978242 -0.         -0.          0.        ]
Lasso training score:  0.49990907988479444
Lasso test score:  0.5410461991374984


#Ridge Regression/L2 Regularization

In [ ]:
#Reduce effects of all coefficients/collinearity with Ridge Regression
from sklearn.linear_model import Ridge

ridge_regression = Ridge (alpha=10) #alpha is a hyperparameter. Increasing it increases regularization

features_standardized_train, features_standardized_test, target_train, target_test = train_test_split(features_standardized, target, test_size=0.25, random_state=0)
ridge_model = ridge_regression.fit(features_standardized_train, target_train)

print("Ridge model intercept:", ridge_model.intercept_)
print("Ridge model coefficients:", ridge_model.coef_)

print("Ridge training score: ", ridge_model.score(features_standardized_train, target_train))
print("Ridge test score: ", ridge_model.score(features_standardized_test, target_test))

Ridge model intercept: [0.19136873]
Ridge model coefficients: [[ 0.83896426  0.05825698 -0.06185699  0.05429043]]
Ridge training score:  0.5165309187044727
Ridge test score:  0.5219382795294056
